In [1]:
import numpy as np
import random

In [2]:
def gettrees(name):
    file = open(name+".trees","r")
    lines = file.readlines()
    trees = []
    tree = []
    for line in lines:
        if not line.strip(): # empty line
            trees.append(tree)
            tree=[]
            continue
        firstchar = line.strip()[0]
        if firstchar == '-': # end of solution
            break
        tree.append(line.strip())
    file.close()
    for t,tree in enumerate(trees):
        processedtree = []
        for l,layer in enumerate(tree[:-1]):
            processedtree.append([x.strip() for x in layer.split(',') if x.strip()])
        processedtree.append(tree[-1].strip().split())
        trees[t]=processedtree
    return trees

def str2int(s):
    return [[int(c) for c in s1] for s1 in s.split()]

def product(a, b, c):
    a = str2int(a)
    b = str2int(b)
    c = str2int(c)
    for g in range(len(a)):
        if a[g]==c[g] and sum(b[g])==0:
            continue
        split = -1
        for i in range(1,len(a[g])):
            if a[g][:i]==b[g][:i] and a[g][i:]==c[g][i:] and b[g][i]==c[g][i-1] and sum(b[g][i+1:])==0 and sum(c[g][:i-1])==0:
                split=i
                break
        if split>-1:
            continue
        if a[g]==b[g] and sum(c[g])==0:
            continue
        return False
    return True

def ruleindex(layer1,layer2):
    for n in range(len(layer1)):
        if layer1[:n]+layer1[n+1:]==layer2[:n]+layer2[n+2:]:# and product(layer1[n],layer2[n],layer2[n+1]):
            return n
    return -1

In [3]:
def getcsg(triples,rules,counts,textfile,name,solnum=0):
    trees = gettrees(name)
    starttriples=[]
    startcounts=[]
    for d in range(len(trees)):
        tree = trees[d]
        ruleindices = [ruleindex(tree[l],tree[l+1]) for l in range(len(tree)-2)]
        if -1 in ruleindices:
            continue
        if ["L",tree[0][0],"R"] not in starttriples:
            starttriples.append(["L",tree[0][0],"R"])
            startcounts.append(1)
        else:
            startcounts[starttriples.index(["L",tree[0][0],"R"])]+=1
        for l in range(len(tree)-2):
            n = ruleindices[l]
            if n==-1:
                print(d,l)
            if n==0:
                left = "L"
            else:
                left = tree[l][n-1]
            if n==l:
                right = "R"
            else:
                right = tree[l][n+1]
            triple = [left,tree[l][n],right]
            rule = [tree[l+1][n],tree[l+1][n+1]]
            if triple not in triples:
                triples.append(triple)
                rules.append([rule])
                counts.append([1])
            else:
                t = triples.index(triple)
                if rule not in rules[t]:
                    rules[t].append(rule)
                    counts[t].append(1)
                else:
                    r = rules[t].index(rule)
                    counts[t][r]+=1
        for n in range(len(tree[-1])):
            if n==0:
                left = "L"
            else:
                left = tree[-2][n-1]
            if n==len(tree[-2])-1:
                right = "R"
            else:
                right = tree[-2][n+1]
            triple = [left,tree[-2][n],right]
            rule = [tree[-1][n]]
            if triple not in triples:
                triples.append(triple)
                rules.append([rule])
                counts.append([1])
            else:
                t = triples.index(triple)
                if rule not in rules[t]:
                    rules[t].append(rule)
                    counts[t].append(1)
                else:
                    r = rules[t].index(rule)
                    counts[t][r]+=1
    prob=[]
    for t in range(len(triples)):
        tot=sum(counts[t])
        p=[]
        for r in range(len(rules[t])):
            p.append(counts[t][r]/tot)
        prob.append(p)
    #compute perplexity
    synlogsum = 0
    lexlogsum = 0
    sentencecount = 0
    wordcount = 0
    for d in range(len(trees)): # loop over every sentence in the data set
        tree = trees[d]
        if "x" in tree[0][0]: # this means the algorithm did not find a grammatical parse for this sentence
            continue
        ruleindices = [ruleindex(tree[l],tree[l+1]) for l in range(len(tree)-2)]
        if -1 in ruleindices:
            continue
        sentencecount+=1
        wordcount+=len(tree[-1])
        s = starttriples.index(["L",tree[0][0],"R"])
        synlogsum += np.log(startcounts[s]/sum(startcounts))
        for l in range(len(tree)-2): # every layer in the sentence's parse tree
            n = ruleindices[l]
            if n==0:
                left = "L"
            else:
                left = tree[l][n-1]
            if n==l:
                right = "R"
            else:
                right = tree[l][n+1]
            triple = [left,tree[l][n],right]
            s = triples.index(triple)
            r = rules[s].index(tree[l+1][n:n+2])
            synlogsum += np.log(counts[s][r]/sum(counts[s]))
        l = len(tree)-1
        for n in range(len(tree[-1])): # now look at the top layer and the words in the sentence
            if n==0:
                left = "L"
            else:
                left = tree[-2][n-1]
            if n==len(tree[-2])-1:
                right = "R"
            else:
                right = tree[-2][n+1]
            triple = [left,tree[-2][n],right]
            s = triples.index(triple)
            r = rules[s].index([tree[-1][n]])
            lexlogsum += np.log(counts[s][r]/sum(counts[s]))
    synppl = np.exp(-synlogsum/(wordcount-sentencecount))
    lexppl = np.exp(-lexlogsum/wordcount)
    #return everything
    return [starttriples,startcounts,triples,rules,counts,prob,synppl,lexppl]    

def writecsg(starttriples,startcounts,triples,rules,counts,start="",restarts=10,solnum=0):
    tree=[]
    plist=[]
    if start=="":
        rand = random.random()
        tot = sum(startcounts)#len(startcounts)#
        p = 0
        for s in range(len(starttriples)):
            p+=startcounts[s]/tot#1/tot#
            if p>rand:
                start = starttriples[s]
                break
    sentence=["L",start[1],"R"]
    tree.append(sentence[1:-1])
    terminal=[1,0,1]
    words=["."," ","."]
    count=0
    while " " in words:
        options=0
        for n in range(1,len(sentence)-1):
            triple = sentence[n-1:n+2]
            if words[n]==" " and triple in triples:
                count+=1
                options+=1
                rand = random.random()
                t = triples.index(triple)
                tot = sum(counts[t])#len(counts[t])#
                p = 0
                for r in range(len(rules[t])):
                    p+=counts[t][r]/tot#1/tot#
                    if p>rand:
                        if len(rules[t][r])==2:
                            sentence = sentence[:n]+rules[t][r]+sentence[n+1:]
                            words = words[:n]+[" "," "]+words[n+1:]
                            #print(sentence,words)
                            tree.append(sentence[1:-1])
                            plist.append(counts[t][r]/tot)
                        else:
                            words = words[:n]+rules[t][r]+words[n+1:]
                            #print(sentence,words)
                        break
        if options==0 or len(sentence)>20:
            if restarts==0:
                print("          stuck (%d), giving up" % len(sentence),sentence)
                return "",tree,words[1:-1],plist
            #print("          stuck (%d), restarting" % len(sentence),sentence)
            #print(str(count)+": ")
            #print(sentence)
            #print(words)
            return writecsg(starttriples,startcounts,triples,rules,counts,start=start,restarts=restarts-1)
    string = ""
    for w in words[1:-1]:
        string += " " + w
    #string += " ."
    #print(string)
    #return [tree,words[1:-1],plist]
    return string,tree,words[1:-1],plist

In [6]:
textfile = "coca/fic"
name = "test"


file = open(textfile+".txt","r")
datasentences = file.readlines()
datasentences = [x.strip() for x in datasentences]
file.close()

csgpp = []

for sn in range(1):
    print("solution",sn)
    print()

    if True:
        triples = []
        rules = []
        counts = []
        [starttriples,startcounts,triples,rules,counts,prob,synppl,lexppl] = getcsg(triples,rules,counts,textfile,name+str(sn))
        print("perplexity",np.sqrt(synppl*lexppl),synppl,lexppl)
        for i in range(10):
            string,tree,tokens,plist = writecsg(starttriples,startcounts,triples,rules,counts,solnum=sn)
            string = string.strip()
            if string in datasentences:
                string = " " + str(datasentences.index(string))+": "+string
            print(string)
        print()
        csgpp.append(np.sqrt(synppl*lexppl))
        
csgpp = np.array(csgpp)
print("csg: "+str(np.mean(csgpp))+" +/- "+str(np.std(csgpp)))

solution 0

perplexity 1.5187535245072403 2.0230181563470078 1.1401836710987543
 45: How was I to know ?
 71: Go to the left .
I returned to son !
I went to the results .
 71: Go to the left .
I went to the results .
I killed a son !
 11: I am the king of the Blefuscians !
Back to the results .
I thought of the stars .

csg: 1.5187535245072403 +/- 0.0
